In [14]:
## read the file (.pairs)

In [1]:
import numpy.random as random
import numpy as np
import math
import networkx as nx
from scipy import sparse as sp
import random

In [149]:

## need to fix something


def read_pairs_file(pairfile) :
    
    file = open(pairfile, "r")
    
    edges = []
    weights = []
    
    for line in file :
        nodes = line.split()
        edges.append((nodes[0], nodes[1]))
        
        #weight
        if len(nodes) == 3 :
            weights.append(float(nodes[2]))
        else :
            weights.append(1.0)
    file.close()
    
    pair_dict = {}
    t_num = 0 
    
    for edge in edges :
        if edge[0] not in pair_dict :
            pair_dict[edge[0]] = t_num
            t_num +=1
        if edge[1] not in pair_dict :
            pair_dict[edge[1]] = t_num
            t_num +=1
    
    
    adj =[]
    
    adj.extend([[pair_dict[edge[0]],pair_dict[edge[1]]] for edge in edges])
    adj.extend([[pair_dict[edge[1]],pair_dict[edge[0]]] for edge in edges])
    
    print(adj)
    adj = np.array(adj).transpose()
    
    weights.extend(weights)
    w_mat = np.array(weights)
    
    n = t_num  # number of nodes
    #print(n)
    #print(w_mat)
    #print(adj)
    
    A_mat = sp.csr_matrix((w_mat,adj), shape = (n,n))
    
    # number of nodes 
    n_A = A_mat
    sumit  = A_mat.sum()
    m = (sumit/2) # number of edge
    return n,n_A ,m

## we need network

##read from matrix file

def read_adj_file(adjfile) :
    #read matrix file 
    
    file = open(adjfile, 'r')
    cols = file.readline().split()
    
    #number of nodes
    n = len(cols)
   
    #generanate NXN matrix
    mat = sp.lil_matrix((n, n))
    
    
    #splited first row 
    for idx in range(n):
        
        weight = float(cols[idx])
        if weight != 0 :
            mat[0,idx] = weight
    
    #rest of matrix 
    
    r = 1
    for line in file :
        rows= line.split()
        for idx in range(n) :
            weight = float(rows[idx])
            if weight !=0  :
                mat[r,idx] = weight
            
        r +=1
        
    mat = (mat +matrix.T)/2
    
    #number of edges
    
    
    file.close()
    
    m = (mat.sum())/2
    
    return n, mat, m
        

In [154]:
n, n_A, m=read_adj_file("karate.txt")

In [155]:
n_A.toarray()

array([[0., 1., 1., ..., 1., 0., 0.],
       [1., 0., 1., ..., 0., 0., 0.],
       [1., 1., 0., ..., 0., 1., 0.],
       ...,
       [1., 0., 0., ..., 0., 1., 1.],
       [0., 0., 1., ..., 1., 0., 1.],
       [0., 0., 0., ..., 1., 1., 0.]])

In [93]:
#n is number of elements, cn, constrainumber
def initial_grouping(n, cn):

    #get list of nodes
    nodes = [node for node in range(0,n)]
    
    #assign group number to each nodes
    group_assign = [random.randrange(0,cn) for g_num in range(len(nodes))]
    
    return nodes, group_assign

#partition is group-assigned list
def partition_matrix(partition) :
    n = len(partition)
    data= np.ones(n)
    #print(data)
    ij = np.array([partition, list(range(0,n))])
    #print(ij)
    grouping_matrix = sp.csr_matrix((data,ij))
    return grouping_matrix

# s = partition matrix, k = degree matrix , m = number of edge 

#generating degree matrix 
def degree_matrix(adj) :
    d_mat = n_A.sum(axis=1)
    return d_mat

#calculate modularity
def get_modularity(adj, s, k, m):
    
    # formula modularity = (Adj * p_matrix - (d_matrix)^2 * p_matrix/2m) / 2m 
    
    adj_partition  = (s* adj *s.transpose()).diagonal().sum()
    deg_par = np.array(s*k)
    degree_partition = np.square(deg_par).sum()/(2*m) 
    
    #modularity
    Q = (adj_partition - degree_partition) /(2*m)
    
    return Q

def patching(group_list, cn) :
    
    n_nodes =len(group_list)
    #cn is constrained number of groups
    n_groups = cn
    #get index and value that want to change
    
    idx = random.choice([x for x in range(0, n_nodes)])
    val = group_list[idx]
    
    n_val = random.choice([y for y in range(0, n_groups) if y != val])
    
    group_list[idx] = n_val
    patched_group = group_list
    
    return patched_group



def annealing_simulating(initial_temp, adj,n, cn,  s, k, m, cooling_constant, iteration) :
    
    svg_group_list = []
    modul_list = []
    old_Q = None
    group_list = None
    for i in range(iteration) :
        
        temp = new_temp(cooling_constant, initial_temp, i)
        acceptance = False 
        num_rejection = -1
        #n numberof nodes, cn constrained number of groups
        rej_threshold = cal_rej_thres(n,cn)
        #print(rej_threshold)
        
        while acceptance is False :
            
            num_rejection +=1
            #print(old_Q)
            
            #when the very first
            if old_Q is None :
                node_list , group_list = initial_grouping(n,cn)
                s = partition_matrix(group_list)
                old_Q = get_modularity(adj, s, k, m)
                
            
            # patching new grouping list
            new_grouping = patching(group_list, cn)
            #print(new_grouping)
            
            # new grouping matrix
            new_s = partition_matrix(new_grouping)
            
            # new modularity with new grouping matrix
            new_Q = get_modularity(adj,new_s,k,m)
            #print(new_Q)
            
            acceptance = check_acceptance(old_Q, new_Q, temp)
            
            #print(acceptance)
            #print(num_rejection)
            if num_rejection > rej_threshold:
                break
            
            
            if acceptance is True :
                
                #if new_grouping in svg_group_list :
                    #acceptance = False
                #else :
                svg_group_list.append(new_grouping)
                modul_list.append(new_Q)
                #print(modul_list[-1])
                old_Q = new_Q
                group_list = new_grouping
                
    opt = max(modul_list)
    idx_opt = modul_list.index(opt)
    
    opt_group = svg_group_list[idx_opt]
    
    opt_grouping = svg_format(opt_group) 
    opt_Q = opt
    
    return opt_grouping, opt_Q, svg_group_list, modul_list
            


# get updated temperature
def new_temp(cooling_constant, prev_temp, times) :
    new_temp = (math.pow(cooling_constant,times)) * prev_temp
    return new_temp 

# check acceptance between old groups and new groups
def check_acceptance(modul_old, modul_new  , temp) :
    if modul_new > modul_old :
        return True
    else :
        new_accept = math.exp((modul_new - modul_old) * temp)
        #modul_new- modul_old is negative value
        # when temp is higher , new_accept is getting smaller 
    return (np.random.random_sample() < new_accept)  #get true or false



#calculate rejection threshold 

def cal_rej_thres(n, nc):
    
    num_pathing = n*nc
    
    num_of_moves = num_pathing
    
    confi = 1 - math.log(0.05)/math.log(num_of_moves)
    
    return confi *num_of_moves * math.log(num_of_moves)

#saving format for check group easily 

def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set



In [97]:
#n, n_A, m=read_pairs_file("karate.pairs")
n,n_A,m = create_from_matrix_file('karate.txt')
k = degree_matrix(n_A)
#(initial_temp, adj, s, k, m, cooling_constant, iteration)
opt_grouping, opt_Q, saving_group, modul_list = annealing_simulating(initial_temp = 40, adj = n_A,n =n, cn=2,  s =s, k =k,m= m, cooling_constant = 1.0015, iteration =10000) 

[[0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [0, 10], [0, 11], [0, 12], [0, 13], [0, 14], [0, 15], [0, 16], [1, 2], [1, 3], [1, 7], [1, 12], [1, 13], [1, 14], [1, 15], [1, 17], [2, 3], [2, 7], [2, 8], [2, 18], [2, 12], [2, 19], [2, 20], [2, 21], [3, 7], [3, 11], [3, 12], [4, 6], [4, 9], [5, 6], [5, 9], [5, 22], [6, 22], [8, 17], [8, 21], [8, 23], [18, 23], [12, 23], [24, 21], [24, 23], [25, 21], [25, 23], [26, 21], [26, 23], [14, 23], [27, 21], [27, 23], [28, 21], [28, 23], [29, 30], [29, 19], [29, 31], [29, 21], [29, 23], [32, 30], [32, 19], [32, 16], [30, 16], [33, 31], [33, 23], [19, 23], [20, 16], [20, 23], [31, 21], [31, 23], [17, 21], [17, 23], [16, 21], [16, 23], [21, 23], [1, 0], [2, 0], [3, 0], [4, 0], [5, 0], [6, 0], [7, 0], [8, 0], [9, 0], [10, 0], [11, 0], [12, 0], [13, 0], [14, 0], [15, 0], [16, 0], [2, 1], [3, 1], [7, 1], [12, 1], [13, 1], [14, 1], [15, 1], [17, 1], [3, 2], [7, 2], [8, 2], [18, 2], [12, 2], [19, 2], [20, 2], [21, 2], [7, 3], [1

0.04922748191978963
0.041337935568704756
0.058842866535174246
0.05843195266272188
0.06632149901380667
0.05843195266272188
0.038708086785009824
0.04068047337278102
0.04996712689020382
0.05687047994740305
0.04725509533201844
0.0680473372781065
0.024326101249178177
0.012491781722550908
0.01282051282051282
0.017422748191978945
-0.01783366206443131
-0.012573964497041455
-0.028928336620644302
-0.06015779092702171
-0.04413214990138069
-0.045693622616699514
-0.0651709401709402
-0.06977317554240632
-0.05950032873109798
-0.01282051282051282
-0.013149243918474732
0.024326101249178177
-8.218277449045512e-05
-0.008629191321498971
0.02753122945430638
0.037393162393162364
0.011505588428665356
-0.0032873109796186604
0.07560815253122946
0.06311637080867846
0.04068047337278102
0.03344838921761997
0.03122945430637741
0.05119986850756083
0.07289612097304407
0.06278763971071664
0.09590729783037479
0.12417817225509535
0.1024819197896121
0.08842866535174228
0.07684089414858647
0.08407297830374752
0.051199868

In [96]:
saving_group[-1]

[1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0]

In [34]:
[1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0]
a = [0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,1,0,0,1,0,1,0,1,1,1,1,1,1,1,1,1,1,1,1]

NameError: name 'opt_Q' is not defined

In [ ]:
def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set

In [22]:
import random
np.random.random_sample()

0.9424695009098216

In [ ]:
def annealing_simulating(self, initial_temp, cooling_constant, iteration) :
    
    svg_group_list = []
    modul_list = []
    old_Q = None
    
    for i in range(iteration) :
        
        temp = new_temp(initial_temp, i)
        acceptance = False 
        
        while acceptance is False :
            
            if old_Q is None :
                old_Q = get_modularity(adj, s, k, m)
            
            new_grouping = patching(group_list, cn)
            
            # new grouping list
            new_s = partition_matrix(new_grouping)
            # new modularity with new grouping list
            new_Q = get_modularity(adj,new_s,k,m)
            
            acceptance = check_acceptance(old_Q, new_Q, temp)
            
            if acceptance is True :
                
                if new_grouping in svg_group_list :
                    acceptance = False
                else :
                    svg_group_list.append(new_grouping)
                    modul_list.append(new_Q)
                    old_Q = new_Q
    
    opt_grouping = svg_format(new_grouping) 
    opt_modul = new_Q
    
    return opt_grouping, opt_modul
            
        
        

In [ ]:
## we're doing lnversely
initial_temp = 40 
cooling_constant = 1.0015

modul_old = 0.335
modul_new = 0.230

## max number of iteration is 10000

for i in range(10000) :
    
    modul_old 
    temp = new_temp(initial_temp, i)
    
    
    ##why we need rejection threshold?

   
    acceptance = False
    while acceptance is False :
        
        if no modularity, old modularity is None  :
            get modularty from initial/grouping 
            new modularity -> old modularity
            
          
        
        point patch = get new partition
        and get new modularity with new partition 
        new temp
        
        ## we only need point pitch since we constrained number of groups
    
        get_modularity of new grouping
        
        check accpetance 
        acceptance = check_acceptance(modul_old, modul_new, temp)
        
        if acceptance is True:
            
            saving groups in savinv_group_list
            if not saving_group is None :
                if grouping is unique
                 adding up
                else 
                acceptance = False
            
#inverse annealing

In [57]:
math.exp(-0.0002*6) # when number i

0.9988007197120864

In [7]:
# initial grouping -> constrain number of groups

In [17]:
math.pow(1.04,2)

1.0816000000000001

In [43]:
random.random_sample()

0.4299052278854143

In [49]:
def check (num):
    return random.random_sample() < num 

In [50]:
check(1.0)

True

In [58]:
cn  = [1,2,3,4]

In [65]:
cn_rm = cn.remove(2)

ValueError: list.remove(x): x not in list

In [67]:
cn.append(2)

In [68]:
cn

[1, 3, 4, 2]

## Getting S(Partition matrix) 

In [2]:
partition = [0,1,1,0,1,0]

In [78]:
S = n_A.as_matrix()

AttributeError: 'numpy.ndarray' object has no attribute 'as_matrix'

In [5]:
n = len(partition)
data= np.ones(n)
print(data)
ij = np.array([partition, list(range(0,n))])
print(ij)
grouping_matrix = sp.csr_matrix((data,ij))

[1. 1. 1. 1. 1. 1.]
[[0 1 1 0 1 0]
 [0 1 2 3 4 5]]


In [6]:
S = grouping_matrix

In [7]:
S.toarray()

array([[1., 0., 0., 1., 0., 1.],
       [0., 1., 1., 0., 1., 0.]])

In [95]:
S.diagonal().sum()

1.0

In [96]:
np.square(S)

ValueError: dimension mismatch

In [97]:
S

<3x6 sparse matrix of type '<class 'numpy.float64'>'
	with 6 stored elements in Compressed Sparse Row format>

In [106]:
n_A.sum(axis=1).shape

(34,)

In [105]:
np.square([1,2])

array([1, 4])

In [183]:
# s = partition matrix, k = degree matrix , m = number of edge 
m = (sumit/2)

def get_modularity(adj, s, k, m ):
    
    # formula modularity = (Adj * p_matrix - (d_matrix)^2 * p_matrix/2m) / 2m 
    
    adj_partition  = (s* adj *s.transpose()).diagonal().sum()
    deg_par = np.array(s*k)
    degree_partition = np.square(deg_par).sum()/(2*m) 
    
    #modularity
    Q = (adj_partition - degree_partition) /(2*m)
    
    return Q
    

78.0


In [184]:
get_modularity(n_A, g_mat, d, m )

0.08678500986193291

In [180]:
d

matrix([[16.],
        [ 9.],
        [10.],
        [ 6.],
        [ 3.],
        [ 4.],
        [ 4.],
        [ 4.],
        [ 5.],
        [ 3.],
        [ 1.],
        [ 2.],
        [ 5.],
        [ 2.],
        [ 3.],
        [ 2.],
        [ 6.],
        [ 4.],
        [ 2.],
        [ 4.],
        [ 3.],
        [12.],
        [ 2.],
        [17.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 2.],
        [ 5.],
        [ 3.],
        [ 4.],
        [ 3.],
        [ 2.]])

In [11]:
#n is number of elements, cn, constrainumber
def initial_grouping(n,cn):
    
    
    #get list of nodes
    nodes = [node for node in range(0,n)]
    
    #assign group number to each nodes
    group_assign = [random.randrange(0,cn) for g_num in range(len(nodes))]
    
    return nodes, group_assign
    

In [14]:
node_list, group_list = initial_grouping(n, 2)

In [14]:

random.randrange(0,3)

0

In [17]:
nodes = [i+1 for i in range(0,4) ]

In [18]:
nodes

[1, 2, 3, 4]

In [125]:
group_number = [random.randrange(0,3) for g_num in range(len(nodes))]

In [126]:
group_number

[2, 2, 2, 1]

In [127]:
node_list , group_list = initial_grouping(10,3)

In [137]:
node_list

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33]

In [138]:
group_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0]

In [15]:
def partition_matrix(partition) :
    print(partition)
    n = len(partition)
    data= np.ones(n)
    print(data)
    ij = np.array([partition, list(range(0,n))])
    print(ij)
    grouping_matrix = sp.csr_matrix((data,ij))
    return grouping_matrix

In [16]:
group_matrix = partition_matrix(group_list)

[1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[ 1  0  0  0  0  0  0  1  0  1  1  0  0  1  1  0  0  1  0  0  1  0  1  0
   0  0  0  1  0  0  1  0  1  0]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33]]


In [17]:
group_matrix.toarray()

array([[0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1.,
        1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
        0., 1.],
       [1., 0., 0., 0., 0., 0., 0., 1., 0., 1., 1., 0., 0., 1., 1., 0.,
        0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0.,
        1., 0.]])

In [142]:
g_mat = partition_matrix(group_list)

[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0]
[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
[[ 0  0  0  0  0  0  0  1  0  0  0  1  1  0  1  0  1  0  0  1  1  1  0  1
   1  1  0  0  1  0  1  1  1  0]
 [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
  24 25 26 27 28 29 30 31 32 33]]


In [145]:
g_mat.toarray()

array([[1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1.,
        0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 1.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0.,
        1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1.,
        1., 0.]])

In [51]:
group_list.partition_matrix()

AttributeError: 'list' object has no attribute 'partition_matrix'

In [64]:
g_mat.transpose().toarray()

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [65]:
g_mat.toarray()

array([[1., 1., 1., 0., 0., 0., 0., 1., 0., 1.],
       [0., 0., 0., 1., 1., 1., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0.]])

In [74]:
n_A

<34x34 sparse matrix of type '<class 'numpy.float64'>'
	with 156 stored elements in Compressed Sparse Row format>

In [18]:
d = n_A.sum(axis=1)

In [ ]:
def degree_matrix(adj) :
    d_mat = n_A.sum(axis=1)
    return d_mat

In [102]:
second_half = group_matrix *d

In [93]:
sec = np.square(second_half).sum() / (sumit/2)

In [97]:
sec

164.30769230769232

In [95]:
first = (group_matrix*n_A*group_matrix.transpose()).diagonal().sum()

In [96]:
first-sec/(sumit/2)

65.89349112426035

In [103]:
second_half

matrix([[60.],
        [96.]])

In [105]:
group_matrix*n_A*group_matrix.transpose().toarray()

array([[16., 44.],
       [44., 52.]])

In [106]:
group_list

[1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1]

In [147]:
fir = (g_mat*n_A*g_mat.transpose(copy=True)).diagonal().sum()

In [149]:
d_s = np.array(g_mat * d)

In [150]:
sec = np.square(d_s).sum()/(2*(sumit/2))

In [155]:
(fir-sec)/(2*(sumit/2))

0.08678500986193291

In [154]:
fir-sec

13.538461538461533

In [158]:
group_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0]

In [162]:
#saving format

fin_set = {}
for idx, gnum in enumerate(group_list) :
    
    if gnum in fin_set :
        fin_set[gnum].append(idx)
    
    else :
        fin_set[gnum] = [idx]

In [163]:
fin_set

{0: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 13, 15, 17, 18, 22, 26, 27, 29, 33],
 1: [7, 11, 12, 14, 16, 19, 20, 21, 23, 24, 25, 28, 30, 31, 32]}

In [172]:
def svg_format(group_list):
    fin_set ={}
    for idx, gnum in enumerate(group_list):
        if gnum in fin_set :
            fin_set[gnum].append(idx)
        else :
            fin_set[gnum] =[idx]
    return fin_set

In [173]:
final = svg_format(group_list)

In [174]:
print(final)

{0: [0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 13, 15, 17, 18, 22, 26, 27, 29, 33], 1: [7, 11, 12, 14, 16, 19, 20, 21, 23, 24, 25, 28, 30, 31, 32]}


In [211]:
ts = [1,2,2,3,1]

In [187]:
ts[2]=0

In [188]:
ts

[1, 2, 0, 3, 1]

In [204]:
ts = [1,2,3]
val=2

In [210]:
random.choice([x for x in ts if x!=val])

1

In [214]:
idx = random.choice([x for x in range(0,4)])

In [217]:
val = ts[idx]
print(val)

2


In [218]:
n_val = random.choice([x for x in range(0,4) if x!= val])

In [219]:
ts

[1, 2, 2, 3, 1]

In [220]:
ts[idx] = n_val
print(ts)

[1, 2, 0, 3, 1]


In [221]:
a = [[1,0,0], [0,1,0]]

In [223]:
[0,0,1] in a

False

In [1]:
def patching(group_list, cn) :
    
    n_nodes =len(group_list)
    #cn is constrained number of groups
    n_groups = cn
    #get index and value that want to change
    
    idx = random.choice([x for x in range(0, n_nodes)])
    val = group_list[idx]
    
    n_val = random.choice([y for y in range(0, n_groups) if y != val])
    
    group_list[idx] = n_val
    patched_group = group_list
    
    return patched_group
    

In [24]:
cn = 2
group_list
patched_group = patching(group_list, cn)

In [23]:
patched_group

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0]

In [25]:
patched_group

[1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0]